In [1]:
from qrisp import QuantumVariable, h, prepare, conjugate
from qrisp.vqe.problems.heisenberg import create_heisenberg_init_function, heisenberg_problem, create_heisenberg_hamiltonian
import networkx as nx
import numpy as np
import scipy
import matplotlib.pyplot as plt
from run_QITE import run_QITE

import pickle
from time import time
from datetime import datetime

def save_data(data, filename):  
    # Open a file for writing
    with open(filename+'.pickle', 'wb') as file:
        # Write the object to the file
        pickle.dump(data, file)

def load_data(filename):
    with open(filename+'.pickle', 'rb') as file:
        # Load the object from the file
        data = pickle.load(file)
    return data

In [2]:
L = 10
G = nx.Graph()
G.add_edges_from([(k,(k+1)%L) for k in range(L-1)]) 
# nx.draw(G, with_labels=True)

In [3]:
from qrisp.operators import X, Y, Z

J = 1
B = 0.5
H = create_heisenberg_hamiltonian(G, J, B)
print(H)

# Define scaling factor
F = 10

# Hamiltonian simulation via first order Suzuki-Trotter formula with 5 steps
def exp_H(qv, t):
    H.trotterization(method='commuting')(qv,t/F,5)

X(0)*X(1) + X(1)*X(2) + X(2)*X(3) + X(3)*X(4) + X(4)*X(5) + X(5)*X(6) + X(6)*X(7) + X(7)*X(8) + X(8)*X(9) + Y(0)*Y(1) + Y(1)*Y(2) + Y(2)*Y(3) + Y(3)*Y(4) + Y(4)*Y(5) + Y(5)*Y(6) + Y(6)*Y(7) + Y(7)*Y(8) + Y(8)*Y(9) + 0.5*Z(0) + Z(0)*Z(1) + 0.5*Z(1) + Z(1)*Z(2) + 0.5*Z(2) + Z(2)*Z(3) + 0.5*Z(3) + Z(3)*Z(4) + 0.5*Z(4) + Z(4)*Z(5) + 0.5*Z(5) + Z(5)*Z(6) + 0.5*Z(6) + Z(6)*Z(7) + 0.5*Z(7) + Z(7)*Z(8) + 0.5*Z(8) + Z(8)*Z(9) + 0.5*Z(9)


In [4]:
O = H.hermitize().eliminate_ladder_conjugates()
commuting_groups = O.group_up(lambda a, b: a.commute(b))
#for group in commuting_groups:
#    print(group)

def trotter_step_2(qarg, t, steps):
    dt = t / steps
    #
    # 1) Forward half-step
    #
    for com_group in commuting_groups:
        with conjugate(com_group.change_of_basis)(qarg, method="commuting") as diagonal_operator:
            intersect_groups = diagonal_operator.group_up(lambda a, b: not a.intersect(b))
            for intersect_group in intersect_groups:
                for term, coeff in intersect_group.terms_dict.items():
                    term.simulate(
                        -coeff * (dt/2) * (-1),
                        qarg
                    )
    
    #
    # 2) Backward half-step (reverse order)
    #
    for com_group in reversed(commuting_groups):
        with conjugate(com_group.change_of_basis)(qarg, method="commuting") as diagonal_operator:
            intersect_groups = diagonal_operator.group_up(lambda a, b: not a.intersect(b))
            for intersect_group in intersect_groups:
                for term, coeff in intersect_group.terms_dict.items():
                    term.simulate(
                        -coeff * (dt/2) * (-1),
                        qarg
                    )

F=10

def exp_H_2(qv, t, steps=2):
    for i in range(steps):
        trotter_step_2(qv, t/F, steps)

# QITE with manually setup eigenstates

In [5]:
# Manual setup
H_matrix = H.to_array()
eigvals, eigvecs = np.linalg.eigh(H_matrix)
idx = np.argsort(eigvals)
eigvals_sorted = eigvals[idx].real
print('Eigen energies:', eigvals_sorted)
eigvecs_sorted = eigvecs[:,idx].T
eigvecs_sorted = [vec.reshape(-1,1) for vec in eigvecs_sorted]

epsilon = 0.001
alpha = 0.5

results= []

mid_id_ls = [1,2,4]
for mid_id in mid_id_ls:
    results_psi = []
    psi = eigvecs_sorted[10] + alpha*eigvecs_sorted[mid_id] + epsilon*eigvecs_sorted[0]
    psi = (psi/np.linalg.norm(psi)).transpose()[0]

    def U_0(qv):
        prepare(qv, psi,reversed=False)

    qv = QuantumVariable(10)
    U_0(qv)

    results.append(run_QITE(H, U_0, exp_H_2, np.linspace(.01,5.0,20), 4, method='GC', use_statevectors=True))
    results.append(run_QITE(H, U_0, exp_H_2, np.linspace(.01,10.0,20), 0, method='HOPF', use_statevectors=True))
    

Eigen energies: [-17.03214083 -16.72269436 -15.72269436 ...  12.80422607  13.
  14.        ]


In [8]:
# Save benchmark data
current_datetime = datetime.now()
date_time_string = current_datetime.strftime("%m-%d-%H")
save_data(results,'data/benchmarks_manual_2nd_B=0.5_'+date_time_string)